In [2]:
pwd()

'C:\\Users\\aparashar'

In [3]:
import codecs
import glob
import regex as re
import pandas as pd
# for converting wx to utf notation
import wxconv
import numpy as np
from wxconv import WXC

In [4]:
%cd E:\\META\\ltrc

E:\META\ltrc


In [5]:
def getLines(file,splitby):
    with codecs.open(file, 'r', encoding="utf-8") as filer:
                    data = filer.read()
                    data = re.sub('\t+', '\t', data)
    lines = data.split(splitby)
    return lines

In [6]:
def get_sentence(row):
    ret=re.findall('"(.+?)"', row['gloss'])
    return ret
def get_gloss(row):
    ret = []
    split = row['gloss'].split(';')
    for i in split:
        if '"' not in i:
            ret.append(i)
        else:
            break
    return ret

In [7]:
#converting hin-eng linked data to a dataframe
with codecs.open('English_Hindi_Linked_Synsets', 'r', encoding="utf-8") as filer:
                    data = filer.read()
                    data = re.sub('\t+', '\t', data)
with codecs.open('English_Hindi_Linked_Synsets_m.txt', 'w', encoding="utf-8") as filew:
    filew.write(data)
linked_df = pd.read_csv('English_Hindi_Linked_Synsets_m.txt', sep='\t',na_filter = False)
linked_df['synset_words'] = linked_df['synset_words'].str.split(', ')
linked_df['hindi_synset_words'] = linked_df['hindi_synset_words']+','
linked_df['eng_ex'] = linked_df.apply(lambda row: get_sentence(row), axis=1)
linked_df['gloss'] = linked_df.apply(lambda row: get_gloss(row), axis=1)
mask_no_engex = linked_df.eng_ex.apply(lambda x: len(x)==0)
print("Total no. of instances : " + str(len(linked_df)))
print("No. of instances having no english examples: " + str(len(linked_df[mask_no_engex])))
#linked_df.assign(eng_ex = get_sentence(linked_df['gloss'].astype("string"))
linked_df[10:25]

Total no. of instances : 17787
No. of instances having no english examples: 10315


,SNo,synset_id,category,synset_words,hindi_synset_words,hindi_synset_id,hindi_category,gloss,eng_ex
10,11,10069427,NOUN,"[executioner, public_executioner]","जल्लाद,वधिक,वधक,",10016,noun,[an official who inflicts capital punishment i...,[]
11,12,1020936,NOUN,[imitation],"अनुकरण,अनुसरण,अनुगमन,नक़ल,नकल,अनुकार,अनुसार,अन...",1002,noun,[copying (or trying to copy) the actions of so...,[]
12,13,7090019,NOUN,"[verbiage, verbalism]","शब्दजाल,शब्दाडंबर,शब्दाडम्बर,वाक्जाल,",10022,noun,[overabundance of words],[]
13,14,10085970,NOUN,"[fiancee, bride-to-be]","वाग्दत्ता,",10024,noun,[a woman who is engaged to be married],[]
14,15,4058096,NOUN,[reading_room],"वाचनालय,",10025,noun,[a room set aside for reading],[]
15,16,2119971,ADJECTIVE,"[thoughtful, serious-minded]","विचारशील,विचारवान,दीर्घदर्शी,विचारवान्,बुद्धिज...",10026,adjective,[acting with or showing thought and good sense],[a sensible young man]
16,17,150505,ADJECTIVE,"[eloquent, facile, fluent, silver, silver-tong...","वाक्पटु,",10030,adjective,"[expressing yourself readily, clearly, effecti...","[able to dazzle with his facile tongue, silver..."
17,18,10420031,NOUN,"[petitioner, suppliant, supplicant, requester]","वादी,फरियादी,मुद्दई,अभियोक्ता,अभियोगकर्ता,अभिय...",10032,noun,[one praying humbly for something],[a suppliant for her favors]
18,19,1380267,ADJECTIVE,[aerial],"वायव्य,",10033,adjective,[growing in air],[aerial roots of a philodendron]
19,20,1380267,ADJECTIVE,[aerial],"वायव्य,",10033,adjective,[in or belonging to the air or operating (for ...,"[aerial particles, small aerial creatures such..."


In [8]:
# converting hindi synset data to a dataframe
d = {'ID':[],'CAT':[],'CONCEPT':[],'EXAMPLE':[],'SYNSET-HINDI':[]}
for line in getLines('hindi.syns','\n\n'):
    internal_lines = line.split('\n')
    #print(internal_lines)
    if len(internal_lines)>0:
        
        d['ID'].append(internal_lines[0].split('::')[1].strip())
        d['CAT'].append(internal_lines[1].split('::')[1].strip())
        d['CONCEPT'].append(internal_lines[2].split('::')[1].strip())
        d['EXAMPLE'].append(internal_lines[3].split('::')[1].strip())
        d['SYNSET-HINDI'].append(internal_lines[4].split('::')[1].strip())

hn_syns_df = pd.DataFrame.from_dict(d)
print(len(hn_syns_df))
hn_syns_df[:10]

38759


,ID,CAT,CONCEPT,EXAMPLE,SYNSET-HINDI
0,1,ADJECTIVE,जिसने जन्म न लिया हो,"""देवकी के अजन्मे बालकों के विषय में भविष्यवाणी...",अजन्मा:अजात:अनुत्पन्न:अनुद्भूत:अप्रादुर्भूत:अज...
1,2,ADJECTIVE,जो शुभ न हो,"""बिल्ली के द्वारा रास्ता काटा जाना अशुभ माना ज...",अशुभ:अमांगलिक:अमाङ्गलिक:मनहूस:अमंगल:अमङ्गल:अक्...
2,3,ADJECTIVE,जो प्रविष्ट न हुआ हो,"""अप्रविष्ट अतिथियों को शीघ्र ही भीतर प्रवेश कर...",अप्रविष्ट
3,4,NOUN,वह स्थान जो पवित्र माना जाता हो,"""हिंदुओं के लिए काशी एक पवित्र स्थान है ।""",पवित्र_स्थान:चैत्य_स्थान:पुण्य_भूमि:पुण्य-स्थल...
4,5,NOUN,वह मंदिर जिसमें भगवान शिव की मूर्ति स्थापित की...,"""वह प्रत्येक सोमवार को शिवालय जाता है ।""",शिवालय:शिव_मंदिर:शिव_मन्दिर:शिवाला:सिवाला:सौधा...
5,6,NOUN,वह स्थान जो पवित्र न हो,"""धार्मिक मान्यता है कि भूत-प्रेत अपवित्र स्थान...",अपवित्र_स्थान:अपुण्यभूमि:अपवित्रस्थली
6,7,ADJECTIVE,जो आया हुआ हो,"""आगत व्यक्तियों का स्वागत करो ।""",आगत:समागत
7,8,ADJECTIVE,जो पैदा हुआ हो या जिसने जन्म लिया हो,"""जन्मे प्राणियों की मृत्यु निश्चित है ।""",जन्मा:उत्पन्न:जन्मा_हुआ:जात:संजात:जनित:प्रसूत:...
8,9,NOUN,ऐसा कार्य जो नीतिपरक हो,"""नैतिक कार्यों के द्वारा ही हम समाज का उत्थान ...",नैतिक_कार्य:सत्कर्म:सुकर्म:अच्छा_काम:सत्कार्य:...
9,10,ADJECTIVE,जिसकी उत्पत्ति हुई हो या जो उगा हो,"""भारत में उत्पन्न चाय अधिक मात्रा में विदेशों ...",उत्पन्न:पैदा_हुआ:पैदा:उपजा:उपजा_हुआ:उतपन्न


In [10]:
# converting mrs file into a dataframe
d ={'conc_label_wx':[],'conc_label_utf':[],'conc_in_eng':[],'mrs_conc':[]}
con = WXC(order='wx2utf', lang='hin')
for line in getLines('H_concept-to-mrs-rels.dat','\n'):
    internal_lines = line.split('\t')
    if len(internal_lines)>=4:
        d['conc_label_wx'].append(internal_lines[1])
        d['conc_label_utf'].append(con.convert(internal_lines[1]))
        d['conc_in_eng'].append(internal_lines[2])
        d['mrs_conc'].append(internal_lines[3])
    else:
        print(internal_lines[2])
        d['conc_label_wx'].append(internal_lines[1])
        d['conc_label_utf'].append(con.convert(internal_lines[1]))
        if len(internal_lines[2])>20:
            d['conc_in_eng'].append(internal_lines[2].split(' ')[0])
            d['mrs_conc'].append(internal_lines[2].split(' ')[1])
        else:
            d['conc_in_eng'].append(internal_lines[2][0:7])
            d['mrs_conc'].append(internal_lines[2][8:])
mrs_df = pd.DataFrame.from_dict(d)
print( "No. of instances in mrs file : "+ str(len(mrs_df)))
mrs_df[200:220]


foundation+stone_1 _foundation_n_1+_stone_n_1)
dutch_1_dutch_a_1_1)
shrug_3_shrug_n_1_1)
No. of instances in mrs file : 54329


,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
200,aBiprAya_6,अभिप्राय_6,purpose_1,_purpose_n_1)
201,aBiprAyAnusAra_1,अभिप्रायानुसार_1,apropos_1,_apropos_a_1)
202,aBipreraNa_1,अभिप्रेरण_1,motivation_1,_motivation_n_for)
203,aBiprewa_1,अभिप्रेत_1,intended_1,_intend_v_for)
204,aBiprewa_2,अभिप्रेत_2,implied_2,_imply_v_1)
205,aBiprewa_3,अभिप्रेत_3,designed_1,_design_v_1)
206,aBIpsiwa_1,अभीप्सित_1,desired_4,_desire_v_1)
207,aBIpsiwa_2,अभीप्सित_2,cherished_3,_cherish_v_1)
208,aBipuRta_1,अभिपुष्ट_1,affirmed_1,_affirm_v_1)
209,aBipuRtapoRaka_1,अभिपुष्टपोषक_1,affirmant_1,_affirmant/NN_u_unknown)


In [15]:
def get_fin_df(row,d,ign):
    conc_label_hn = row['conc_label_utf']
    conc_eng = row['conc_in_eng']
    hn_label = conc_label_hn.split('_')[0]+','
    eng_label = conc_eng.split('_')[0]
    # checking ig the hindi label is present in the hindi synset words list of linked_df
    mask = linked_df.hindi_synset_words.apply(lambda x: hn_label in x)
    # checking if the eng label is present in the eng sysnet word list of linked_df
    mask1 = linked_df.synset_words.apply(lambda x: eng_label in x)
    # taking intersection of both the conditions
    cross = linked_df[mask & mask1]
    cross = cross.reset_index()  
    # if this pair of [hindi_label and conc_eng] are found in the same row of linked_df
    if cross.empty == False:
        
        for index, row in cross.iterrows():
            if (row['synset_id'] not in d['eng_id']) and (row['hindi_synset_id'] not in d['hin_id']):
                # setting a flag signifying that the hindi word and conc eng pair were found in the same row
                d['ign'].append(False)
                d['hin'].append(conc_label_hn)
                d['eng'].append(conc_eng)
                d['eng_id'].append(row['synset_id'])
                d['hin_id'].append(row['hindi_synset_id'])
                d['eng_gloss'].append(row['gloss'])
                d['eng_ex'].append(row['eng_ex'])
                if len(hn_syns_df[hn_syns_df['ID']==row['hindi_synset_id']]) != 0:
                    # getting hindi example and concept from the hindi synset dataframe
                    d['hin_ex'].append(hn_syns_df[hn_syns_df['ID']==row['hindi_synset_id']].EXAMPLE.item())
                    d['hin_gloss'].append(hn_syns_df[hn_syns_df['ID']==row['hindi_synset_id']].CONCEPT.item())
                else:
                    # if the hindi synset id is not found in hindi synset df. then adding empty strings
                    d['hin_ex'].append("")
                    d['hin_gloss'].append("")
    
    # else taking the corresponding hindi and english details(gloss,id, examples) separately in a many to many fashion
    else:
        
        # adding hindi-word and eng_concept pairs not found in the intersection to the ign dict
        ign['hindi_ig'].append(conc_label_hn)
        ign['eng_conc_ig'].append(conc_eng)
        # applying mask and mask1 separately and taking the respective gloss and examples for hindi and english
        hin_masked = linked_df[mask]
        eng_masked = linked_df[mask1]
        for index, row in hin_masked.iterrows():
            for index1,row1 in eng_masked.iterrows():
                # setting a flag signifying that the pair was not found in the same row
                d['ign'].append(True)
                # taking the hindi details from the filtered by <hindi label> df
                d['hin'].append(conc_label_hn)
                d['hin_id'].append(row['hindi_synset_id'])
                
                if len(hn_syns_df[hn_syns_df['ID']==row['hindi_synset_id']]) != 0:
                        # getting hindi example and concept from the hindi synset dataframe
                        d['hin_ex'].append(hn_syns_df[hn_syns_df['ID']==row['hindi_synset_id']].EXAMPLE.item())
                        d['hin_gloss'].append(hn_syns_df[hn_syns_df['ID']==row['hindi_synset_id']].CONCEPT.item())
                else:
                    # if the hindi synset id is not found in hindi synset df. then adding empty strings
                    d['hin_ex'].append("")
                    d['hin_gloss'].append("")
                # taking the eng details from the filtered by its <corrresponding eng meaning/concept word> df
                d['eng'].append(conc_eng)
                d['eng_id'].append(row1['synset_id'])
                d['eng_gloss'].append(row1['gloss'])
                d['eng_ex'].append(row1['eng_ex'])
                

In [22]:
# checking which hindi and englidh words from mrs are not present in the synsets at all
def notFound(row,hin_nf,eng_nf):
    conc_label_hn = row['conc_label_utf']
    conc_eng = row['conc_in_eng']
    hn_label = conc_label_hn.split('_')[0]+','
    hn_label = hn_label.replace('+',' ')
    eng_label = conc_eng.split('_')[0]
    # checking ig the hindi label is present in the hindi synset words list of linked_df
    mask = linked_df.hindi_synset_words.apply(lambda x: hn_label in x)
    # checking if the eng label is present in the eng sysnet word list of linked_df
    mask1 = linked_df.synset_words.apply(lambda x: eng_label in x)
    hin_masked = linked_df[mask]
    eng_masked = linked_df[mask1]
    if hin_masked.empty == True:
        hin_nf.append(hn_label)
    if eng_masked.empty == True:
        eng_nf.append(eng_label)
    

In [ ]:
hin_nf, eng_nf = [],[]
mrs_df.apply(lambda row: notFound(row,hin_nf,eng_nf), axis=1)

In [45]:
print(hin_nf[100:120])
print(eng_nf[100:120])

['अक्षाश,' 'अक्षुब्ध,' 'अक्षौहिणी,' 'अखरोट आदि का छिलका,' 'अखलाक़,'
 'अगम्यता,' 'अगम्यागमन,' 'अगर-मगर,' 'अगर्हित,' 'अगल-बग़ल,'
 'अगाड़ीपिछाड़ी,' 'अगुवाई कर,' 'अग्नि का प्रकाश,' 'अग्नि परीक्षा,'
 'अग्नि सम्बन्धी,' 'अग्नित्र्कीड़ा,' 'अग्निपूजक,' 'अग्निपूजा,' 'अग्निमय,'
 'अग्निसंस्कार,']
['Scientist' 'Seth' 'Shahjahan' 'Singh' 'Sir' 'Sovereignty' 'Spiritualism'
 'Utopia' 'Vaisakh' 'Vaishya' 'Yoga' 'Yogi' 'Zoology' 'Zoroastrian' 'a'
 'aasmus' 'abandonable' 'abandoner' 'abandoning' 'abase']


In [42]:
x=np.array(hin_nf)
hin_nf = np.unique(x)
hindi_nf_df = pd.DataFrame({'hindi_word': hin_nf[:]})

y = np.array(eng_nf)
eng_nf = np.unique(y)
eng_nf_df = pd.DataFrame({'eng_word': eng_nf[:]})
eng_nf_df
# saving hindi not found(hindi_nf) and eng not found(eng_nf) as a csv
eng_nf_df.to_csv('eng_nf.csv', sep ='\t')
hindi_nf_df.to_csv('hin_nf.csv', sep ='\t')


In [16]:
d = {'hin' : [],'eng' : [], 'hin_id':[],'eng_id':[], 'hin_gloss':[],'eng_gloss':[], 'hin_ex':[],'eng_ex':[],'ign':[]}
ign = {'hindi_ig' :[], 'eng_conc_ig':[]}
mrs_df.apply(lambda row: get_fin_df(row,d,ign), axis=1)
# final dataframe depending on the hindi label and eng conc from mrs
fin_df = pd.DataFrame.from_dict(d)
#ignored pairs which were not found in eng-hindi linked df
ign_df = pd.DataFrame.from_dict(ign)

In [47]:
mask_no_engex = fin_df.eng_ex.apply(lambda x: len(x)==0)
mask_no_hin_id = fin_df.hin_ex.apply(lambda x: x=="")
print("No. of total instances: " + str(len(fin_df)))
print("No. of instances having no corresponding hindi ids :" + str(len(fin_df[mask_no_hin_id])))
print("No. of instances having no english examples :" + str(len(fin_df[mask_no_engex])))
print("No. of ignored pairs from the MRS: " + str(len(ign_df)))
#fin_df[mask_no_hin_id]
fin_df[200:220]

No. of total instances: 268244
No. of instances having no corresponding hindi ids :1405
No. of instances having no english examples :72196
No. of ignored pairs from the MRS: 42256


,hin,eng,hin_id,eng_id,hin_gloss,eng_gloss,hin_ex,eng_ex,ign
200,आ_1,come_1,19906,2006709,मध्य दक्षिण अमरीका का एक देश,[be received],"""अट्ठारह सौ पच्चीस में स्पेन से स्वतंत्रता प्र...",[News came in of the massacre in Rwanda],True
201,आ_1,come_1,19906,5404336,मध्य दक्षिण अमरीका का एक देश,[the thick white fluid containing spermatozoa ...,"""अट्ठारह सौ पच्चीस में स्पेन से स्वतंत्रता प्र...",[],True
202,आ_1,come_1,19906,723349,मध्य दक्षिण अमरीका का एक देश,"[come to one' s mind, suggest itself]","""अट्ठारह सौ पच्चीस में स्पेन से स्वतंत्रता प्र...",[It occurred to me that we should hire another...,True
203,आ_1,come_1,19906,2729963,मध्य दक्षिण अमरीका का एक देश,"[come under, be classified or included]","""अट्ठारह सौ पच्चीस में स्पेन से स्वतंत्रता प्र...","[fall into a category, This comes under a new ...",True
204,आ_1,come_1,19906,341917,मध्य दक्षिण अमरीका का एक देश,"[come to pass, arrive, as in due course]","""अट्ठारह सौ पच्चीस में स्पेन से स्वतंत्रता प्र...","[The first success came three days later, It c...",True
205,आ_1,come_1,2004,2006709,जिसे अंगीकार किया गया हो या जिसे अपने ऊपर लिया...,[be received],"""उसने अपने अंगीकृत कार्यभार को सहर्ष निपटाया ।""",[News came in of the massacre in Rwanda],True
206,आ_1,come_1,2004,5404336,जिसे अंगीकार किया गया हो या जिसे अपने ऊपर लिया...,[the thick white fluid containing spermatozoa ...,"""उसने अपने अंगीकृत कार्यभार को सहर्ष निपटाया ।""",[],True
207,आ_1,come_1,2004,723349,जिसे अंगीकार किया गया हो या जिसे अपने ऊपर लिया...,"[come to one' s mind, suggest itself]","""उसने अपने अंगीकृत कार्यभार को सहर्ष निपटाया ।""",[It occurred to me that we should hire another...,True
208,आ_1,come_1,2004,2729963,जिसे अंगीकार किया गया हो या जिसे अपने ऊपर लिया...,"[come under, be classified or included]","""उसने अपने अंगीकृत कार्यभार को सहर्ष निपटाया ।""","[fall into a category, This comes under a new ...",True
209,आ_1,come_1,2004,341917,जिसे अंगीकार किया गया हो या जिसे अपने ऊपर लिया...,"[come to pass, arrive, as in due course]","""उसने अपने अंगीकृत कार्यभार को सहर्ष निपटाया ।""","[The first success came three days later, It c...",True


In [39]:
ign_df[:10]

,hindi_ig,eng_conc_ig
0,आ_1,come_1
1,आब_1,lustre_1
2,आब_2,brilliance_1
3,आभा_2,splendour_1
4,आभा_3,tint_1
5,आभा_4,tinge_1
6,अभागा_2,accursed_1
7,अभागा_3,unlucky_1
8,अभागिन_1,accursed_2
9,आबकारी_1,excise_1


In [43]:
fin_df.to_csv('final_linked_set_2.csv', sep ='\t')
